<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Script_Carga_Completa_Todos_instrumentos_CVS_XLSX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
import io

# 🔹 DataFrames independientes para puntos fijos
df_puntos_fijos_mi = pd.DataFrame()
df_puntos_fijos_md = pd.DataFrame()

# 🔹 Diccionario para el resto de instrumentos
datos = {
    "piezometros_electricos": pd.DataFrame(),
    "inclinometros": pd.DataFrame(),
    "asentamiento": pd.DataFrame(),
    "piezometros_casagrande": pd.DataFrame(),
    "freatimetros": pd.DataFrame(),
    "extensometro": pd.DataFrame()
}

# 🔎 Función para detectar el tipo de instrumento
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        return "puntos_fijos"
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# 🔎 Función para detectar el margen (solo puntos fijos)
def detectar_margen(nombre):
    nombre = nombre.lower()
    if "mi" in nombre:
        return "mi"
    elif "md" in nombre:
        return "md"
    return None

# 📂 Widget de carga de archivos
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# 📥 Función de carga principal
def cargar_archivos(change):
    global df_puntos_fijos_mi, df_puntos_fijos_md
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
                    continue

                df.columns = df.columns.str.strip()
                instrumento = detectar_instrumento(nombre_archivo)

                if instrumento == "puntos_fijos":
                    margen = detectar_margen(nombre_archivo)
                    if margen == "mi":
                        df_puntos_fijos_mi = pd.concat([df_puntos_fijos_mi, df], ignore_index=True)
                        print(f"✅ {nombre_archivo} → Puntos Fijos MI")
                    elif margen == "md":
                        df_puntos_fijos_md = pd.concat([df_puntos_fijos_md, df], ignore_index=True)
                        print(f"✅ {nombre_archivo} → Puntos Fijos MD")
                    else:
                        print(f"⚠️ No se detectó margen en: {nombre_archivo}")
                elif instrumento:
                    datos[instrumento] = pd.concat([datos[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento}")
                else:
                    print(f"❌ Instrumento no reconocido: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# 📊 Menú dinámico de visualización
def mostrar_menu():
    opciones = ["puntos_fijos_mi", "puntos_fijos_md"] + list(datos.keys())
    selector = widgets.Dropdown(
        options=opciones,
        description='Instrumento:',
        style={'description_width': 'initial'}
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        print(f"📊 Datos para: {seleccion}")
        if seleccion == "puntos_fijos_mi":
            display(df_puntos_fijos_mi.head())
        elif seleccion == "puntos_fijos_md":
            display(df_puntos_fijos_md.head())
        else:
            display(datos[seleccion].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# 🔗 Conectar el evento de carga
upload_widget.observe(cargar_archivos, names='value')

# 🖥️ Mostrar interfaz inicial
display(upload_widget)
display(output)


FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Datos para: puntos_fijos_md


,FECHA,MARGEN,INSTRUMENTO,DELTA_NORTE_[M],DELTA_ESTE_[M],DELTA_COTA_[M],DISTANCIA_[M],DISTANCIA_(MM),AZIMUT_REF._AL_NORTE,TASA_NORTE_(MM/DIA),TASA_ESTE_(MM/DIA),TASA_COTA_(MM/DIA),TASA_DISTANCIA_(MM/DIA)
0,04/10/2021,Derecha,PF01 PRIMA,0.000333,-0.000667,0.003333,0.000745,0.745356,296.565051,0.041667,-0.083333,0.416667,0.093169
1,10/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.004333,0.000943,0.942809,225.000000,-0.047619,-0.047619,0.309524,0.067344
2,14/10/2021,Derecha,PF01 PRIMA,0.003333,0.003333,0.003333,0.004714,4.714045,45.000000,0.185185,0.185185,0.185185,0.261891
3,18/10/2021,Derecha,PF01 PRIMA,0.002333,0.002333,0.000333,0.003300,3.299832,45.000000,0.106061,0.106061,0.015152,0.149992
4,27/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.005333,0.000943,0.942809,225.000000,-0.021505,-0.021505,0.172043,0.030413


In [19]:
!pip install plotly --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [3]:
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd

# ================================================================
# 🔧 CONFIGURACIÓN ESPECÍFICA PARA GOOGLE COLAB
# ================================================================

# Configurar Plotly para Google Colab
import plotly.io as pio
pio.renderers.default = 'colab'

# Función para mostrar gráficos en Colab (método alternativo)
def mostrar_grafico_colab(fig):
    """Función robusta para mostrar gráficos Plotly en Google Colab"""
    try:
        # Método 1: Usar show() directo (más confiable en Colab)
        fig.show()
    except Exception as e1:
        try:
            # Método 2: Convertir a HTML y mostrar
            html_str = fig.to_html(include_plotlyjs='cdn')
            display(HTML(html_str))
        except Exception as e2:
            try:
                # Método 3: Usar plotly offline
                from plotly.offline import iplot, init_notebook_mode
                init_notebook_mode(connected=True)
                iplot(fig)
            except Exception as e3:
                print(f"❌ Error mostrando gráfico:")
                print(f"   Método 1: {e1}")
                print(f"   Método 2: {e2}")
                print(f"   Método 3: {e3}")
                print("💡 Intenta ejecutar: !pip install plotly --upgrade")

# ================================================================
# 📊 INTERFAZ INTERACTIVA MEJORADA
# ================================================================

# Combinar claves del diccionario + puntos fijos
# Nota: Asegúrate de que estas variables existan en tu entorno
try:
    opciones_instrumentos = ["puntos_fijos_mi", "puntos_fijos_md"] + list(datos.keys())
except NameError:
    print("⚠️ La variable 'datos' no está definida. Usando opciones de ejemplo.")
    opciones_instrumentos = ["puntos_fijos_mi", "puntos_fijos_md", "ejemplo_1", "ejemplo_2"]

# Widget de selección de instrumento con estilo mejorado
selector_instr = widgets.Dropdown(
    options=opciones_instrumentos,
    description="🔧 Instrumento:",
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Widgets para columnas (se rellenan dinámicamente)
selector_x = widgets.Dropdown(
    description='📊 Eje X:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

selector_y = widgets.Dropdown(
    description='📈 Eje Y:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Selector de tipo de gráfico
selector_tipo_graf = widgets.Dropdown(
    options=[
        ('📈 Línea', 'line'),
        ('🔵 Puntos', 'scatter'),
        ('📊 Línea + Puntos', 'lines+markers'),
        ('📉 Área', 'area')
    ],
    value='lines+markers',
    description='📋 Tipo:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='350px')
)

# Botón de graficación mejorado
boton_graf = widgets.Button(
    description="📈 Generar Gráfico",
    button_style='success',
    icon='chart-line',
    layout=widgets.Layout(width='200px', height='40px')
)

# Botón para limpiar output
boton_limpiar = widgets.Button(
    description="🧹 Limpiar",
    button_style='warning',
    icon='trash',
    layout=widgets.Layout(width='120px', height='40px')
)

# Output widget con estilo
out = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

# ================================================================
# 🔄 FUNCIONES DE ACTUALIZACIÓN Y GRAFICACIÓN
# ================================================================

def actualizar_columnas(change):
    """Actualiza las columnas disponibles al cambiar el instrumento"""
    with out:
        clear_output(wait=True)
        seleccion = selector_instr.value

        try:
            # Obtener DataFrame según selección
            if seleccion == "puntos_fijos_mi":
                df = df_puntos_fijos_mi.copy()
            elif seleccion == "puntos_fijos_md":
                df = df_puntos_fijos_md.copy()
            elif seleccion in datos:
                df = datos[seleccion].copy()
            else:
                print(f"⚠️ No se encontraron datos para: {seleccion}")
                return

            if df.empty:
                print("⚠️ El DataFrame seleccionado está vacío.")
                return

            # Actualizar opciones de selectores
            columnas = df.columns.tolist()
            selector_x.options = columnas
            selector_y.options = columnas

            # Seleccionar valores por defecto inteligentes
            if len(columnas) >= 2:
                # Buscar columnas que podrían ser fechas para X
                fecha_cols = [col for col in columnas if any(keyword in col.lower()
                             for keyword in ['fecha', 'date', 'time', 'tiempo'])]
                if fecha_cols:
                    selector_x.value = fecha_cols[0]
                else:
                    selector_x.value = columnas[0]

                # Para Y, seleccionar la segunda columna o una numérica
                numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
                if numeric_cols and len(numeric_cols) > 0:
                    selector_y.value = numeric_cols[0]
                else:
                    selector_y.value = columnas[1] if len(columnas) > 1 else columnas[0]

            # Mostrar vista previa
            print(f"📄 Vista previa de {seleccion}:")
            print(f"📊 Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
            display(df.head())

            # Mostrar info de tipos de datos
            print(f"\n📋 Tipos de datos:")
            tipos_info = df.dtypes.value_counts()
            for tipo, cantidad in tipos_info.items():
                print(f"   {tipo}: {cantidad} columnas")

        except NameError as e:
            print(f"❌ Error: Variable no definida - {e}")
            print("💡 Asegúrate de que las variables df_puntos_fijos_mi, df_puntos_fijos_md y datos estén definidas")
        except Exception as e:
            print(f"❌ Error inesperado: {e}")

def graficar_interactivo(b):
    """Genera el gráfico interactivo con las selecciones actuales"""
    with out:
        seleccion = selector_instr.value
        col_x = selector_x.value
        col_y = selector_y.value
        tipo_graf = selector_tipo_graf.value

        if col_x is None or col_y is None:
            print("⚠️ Por favor selecciona ambas columnas (X e Y)")
            return

        try:
            # Obtener DataFrame
            if seleccion == "puntos_fijos_mi":
                df = df_puntos_fijos_mi.copy()
            elif seleccion == "puntos_fijos_md":
                df = df_puntos_fijos_md.copy()
            elif seleccion in datos:
                df = datos[seleccion].copy()
            else:
                print(f"⚠️ No se encontraron datos para: {seleccion}")
                return

            print(f"🎯 Generando gráfico: {col_y} vs {col_x}")
            print(f"📊 Tipo: {tipo_graf}")
            print(f"📄 Datos: {len(df)} puntos")

            # Crear gráfico según tipo seleccionado
            if tipo_graf == 'area':
                fig = px.area(df, x=col_x, y=col_y,
                             title=f"{seleccion.upper()}: {col_y} vs {col_x}")
            else:
                fig = px.scatter(df, x=col_x, y=col_y,
                               title=f"{seleccion.upper()}: {col_y} vs {col_x}")

                # Configurar tipo de traza
                if tipo_graf == 'line':
                    fig.update_traces(mode='lines')
                elif tipo_graf == 'scatter':
                    fig.update_traces(mode='markers')
                elif tipo_graf == 'lines+markers':
                    fig.update_traces(mode='lines+markers')

            # Mejorar estilo del gráfico
            fig.update_layout(
                width=800,
                height=500,
                title_font_size=16,
                xaxis_title_font_size=14,
                yaxis_title_font_size=14,
                showlegend=True,
                template='plotly_white'
            )

            # Mostrar gráfico con método robusto
            print("📈 Mostrando gráfico...")
            mostrar_grafico_colab(fig)
            print("✅ Gráfico generado exitosamente")

        except NameError as e:
            print(f"❌ Error: Variable no definida - {e}")
        except Exception as e:
            print(f"❌ Error al generar el gráfico: {e}")
            print(f"🔍 Tipo de error: {type(e).__name__}")

            # Información adicional para debugging
            try:
                print(f"📊 Info del DataFrame:")
                print(f"   Columnas: {df.columns.tolist()}")
                print(f"   Tipos de {col_x}: {df[col_x].dtype}")
                print(f"   Tipos de {col_y}: {df[col_y].dtype}")
                print(f"   Valores nulos en {col_x}: {df[col_x].isnull().sum()}")
                print(f"   Valores nulos en {col_y}: {df[col_y].isnull().sum()}")
            except:
                pass

def limpiar_output(b):
    """Limpia el área de output"""
    with out:
        clear_output()
        print("🧹 Área limpia. Selecciona un instrumento para comenzar.")

# ================================================================
# 🔗 CONECTAR EVENTOS
# ================================================================

selector_instr.observe(actualizar_columnas, names='value')
boton_graf.on_click(graficar_interactivo)
boton_limpiar.on_click(limpiar_output)

# ================================================================
# 🎨 CREAR INTERFAZ CON LAYOUT MEJORADO
# ================================================================

# Crear layout organizado
controles_principales = widgets.VBox([
    widgets.HTML("<h3>🔧 Configuración del Gráfico</h3>"),
    selector_instr,
    widgets.HBox([selector_x, selector_y]),
    selector_tipo_graf
])

controles_botones = widgets.HBox([
    boton_graf,
    boton_limpiar
], layout=widgets.Layout(justify_content='flex-start'))

interfaz_completa = widgets.VBox([
    controles_principales,
    controles_botones,
    widgets.HTML("<h3>📊 Resultados</h3>"),
    out
])

# ================================================================
# 🚀 MOSTRAR INTERFAZ
# ================================================================

print("🎯 Interfaz de Visualización Interactiva Cargada")
print("💡 Selecciona un instrumento para comenzar")

# Mostrar la interfaz
display(interfaz_completa)

# Mensaje inicial en el output
with out:
    print("👋 ¡Bienvenido al sistema de visualización!")
    print("📋 Instrucciones:")
    print("   1. Selecciona un instrumento del menú desplegable")
    print("   2. Espera a que se carguen las columnas")
    print("   3. Selecciona las columnas para X e Y")
    print("   4. Elige el tipo de gráfico")
    print("   5. Haz clic en 'Generar Gráfico'")
    print("\n🔧 Si no ves gráficos, verifica que Plotly esté instalado:")
    print("   !pip install plotly --upgrade")

🎯 Interfaz de Visualización Interactiva Cargada
💡 Selecciona un instrumento para comenzar
